# Bert text classification on SST2 using PyTorch





In [1]:
!pip install -r requirements_notebook.txt

In [2]:
import sys, os, shutil
import logging

sys.path.append("src")

logging.basicConfig(level="INFO", handlers=[logging.StreamHandler(sys.stdout)],
                        format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')

### Bucket and role set up

In [3]:
import sagemaker, boto3
from sagemaker import get_execution_role
sm_session = sagemaker.session.Session()


sagemaker_session = sagemaker.Session()
account_id =  boto3.client('sts').get_caller_identity().get('Account')
region = boto3.session.Session().region_name


#role = sagemaker.get_execution_role()
role="arn:aws:iam::{}:role/service-role/AmazonSageMaker-ExecutionRole-20190118T115449".format(account_id)


2021-01-21 19:06:52,364 - sagemaker.analytics - WARNING - pandas failed to import. Analytics features will be impaired or broken.
2021-01-21 19:06:52,467 - botocore.credentials - INFO - Found credentials in shared credentials file: ~/.aws/credentials
2021-01-21 19:06:52,603 - botocore.credentials - INFO - Found credentials in shared credentials file: ~/.aws/credentials
2021-01-21 19:06:52,666 - botocore.credentials - INFO - Found credentials in shared credentials file: ~/.aws/credentials


In [4]:
resume_from_checkpoint=False

In [5]:
data_bucket = sm_session.default_bucket()

data_bucket_prefix = "bert-sst2-classify"

s3_uri_data = "s3://{}/{}/data".format(data_bucket, data_bucket_prefix)
s3_uri_train = "{}/{}".format(s3_uri_data, "train.csv")
s3_uri_val = "{}/{}".format(s3_uri_data, "dev.csv")

s3_uri_test = "{}/{}".format(s3_uri_data, "test.csv")

s3_output_path = "s3://{}/{}/output".format(data_bucket, data_bucket_prefix)
s3_code_path = "s3://{}/{}/code".format(data_bucket, data_bucket_prefix)
s3_checkpoint = "s3://{}/{}/checkpoint".format(data_bucket, data_bucket_prefix)

In [6]:
import boto3

def delete_s3_objects(s3_uri):
    client = boto3.client('s3')
    s3_uri = s3_uri.replace("s3://","")
    bucket, prefix = s3_uri.split("/")[0], "/".join( s3_uri.split("/")[1:])
    
    response = client.list_objects(
    Bucket=bucket,
    Delimiter='|',
    Prefix=prefix,
    MaxKeys = 20
)
    s3 = boto3.resource('s3')
    for item in response.get("Contents", []):
        print("Deleting {}".format(item["Key"]))
        obj = s3.Object(bucket, item["Key"] )
        obj.delete()
    
    


In [7]:
if not resume_from_checkpoint:
    delete_s3_objects(s3_checkpoint)

In [8]:
prepare_dataset = True

## Prepare dataset

In [9]:
raw_data_dir = "tmp"
processed_out_dir = os.path.join(raw_data_dir, "processd")

if os.path.exists(processed_out_dir):
    shutil.rmtree(processed_out_dir)

os.makedirs(processed_out_dir, exist_ok=True)


In [10]:
s3_uri_train

's3://sagemaker-us-east-2-324346001917/bert-sst2-classify/data/train.csv'

In [11]:
from urllib.request import urlopen 
from io import BytesIO
from zipfile import ZipFile
import shutil, glob

def download_glue_sst2(output_dir, glue_sst2_uri="https://dl.fbaipublicfiles.com/glue/data/SST-2.zip"):
    resp = urlopen(glue_sst2_uri)
    zip_file = ZipFile(BytesIO(resp.read()))
    zip_file.extractall(output_dir)
    for f in glob.glob("{}/SST-2/*.tsv".format(output_dir)):
        shutil.move(f, output_dir)
    
    

In [12]:
#from utils.sst2_split_utils import SST2SplitUtils
from s3_util import S3Util

if prepare_dataset:
#     input_file = os.path.join(raw_data_dir , "datasetSentences.txt")
#     split_file = os.path.join(raw_data_dir , "datasetSplit.txt")
#     dictionary = os.path.join(raw_data_dir , "dictionary.txt")
#     label_file = os.path.join(raw_data_dir , "sentiment_labels.txt")
#     SST2SplitUtils().split( input_file, split_file, dictionary, label_file, processed_out_dir)

    
    download_glue_sst2(processed_out_dir)
    S3Util().upload_file(os.path.join(processed_out_dir, "train.tsv"), s3_uri_train )
    S3Util().upload_file(os.path.join(processed_out_dir, "dev.tsv"), s3_uri_val )
    S3Util().upload_file(os.path.join(processed_out_dir, "test.tsv"), s3_uri_test )
    
    

Uploading file tmp/processd/train.tsv to s3://sagemaker-us-east-2-324346001917/bert-sst2-classify/data/train.csv in 11.44324 seconds
Uploading file tmp/processd/dev.tsv to s3://sagemaker-us-east-2-324346001917/bert-sst2-classify/data/dev.csv in 4.87655 seconds
Uploading file tmp/processd/test.tsv to s3://sagemaker-us-east-2-324346001917/bert-sst2-classify/data/test.csv in 5.758917 seconds


## Train

This shows you how to train BERT on SageMaker using SPOT instances

In [13]:
inputs_full =  {
    "train" : s3_uri_train,
    "val" : s3_uri_val,

}



inputs = inputs_full

In [14]:
sm_localcheckpoint_dir="/opt/ml/checkpoints/"

In [15]:
instance_type = "ml.p3.2xlarge"
instance_type_gpu_map = {"ml.p3.8xlarge":4, "ml.p3.2xlarge": 1, "ml.p3.16xlarge":8}

In [16]:
hp = {
"epochs" : 30,
"earlystoppingpatience" : 3,
# Increasing batch size might end up with CUDA OOM error, increase grad accumulation instead
"batch" : 8 * instance_type_gpu_map[instance_type],
"trainfile" :s3_uri_train.split("/")[-1],
"valfile" : s3_uri_val.split("/")[-1],
"datasetfactory":"datasets.sst2_dataset_factory.SST2DatasetFactory",
# The number of steps to accumulate gradients for
"gradaccumulation" : 4,
"log-level":"INFO",
# This param depends on your model max pos embedding size or when large you might end up with CUDA OOM error    
"maxseqlen" : 512,
# Make sure the lr is quite small, as this is a pretrained model..
"lr":0.00001,
# Use finetuning (set to 1), if you only want to change the weights in the final classification layer.. 
"finetune": 0,
"checkpointdir" : sm_localcheckpoint_dir,
# Checkpoints once every n epochs
"checkpointfreq": 2
}



In [17]:
hp

{'epochs': 30,
 'earlystoppingpatience': 3,
 'batch': 8,
 'trainfile': 'train.csv',
 'valfile': 'dev.csv',
 'datasetfactory': 'datasets.sst2_dataset_factory.SST2DatasetFactory',
 'gradaccumulation': 4,
 'log-level': 'INFO',
 'maxseqlen': 512,
 'lr': 1e-05,
 'finetune': 0,
 'checkpointdir': '/opt/ml/checkpoints/',
 'checkpointfreq': 2}

In [18]:
inputs

{'train': 's3://sagemaker-us-east-2-324346001917/bert-sst2-classify/data/train.csv',
 'val': 's3://sagemaker-us-east-2-324346001917/bert-sst2-classify/data/dev.csv'}

In [19]:
metric_definitions = [{"Name": "TrainLoss",
                     "Regex": "###score: train_loss### (\d*[.]?\d*)"}
                    ,{"Name": "ValidationLoss",
                     "Regex": "###score: val_loss### (\d*[.]?\d*)"}
                    ,{"Name": "TrainScore",
                     "Regex": "###score: train_score### (\d*[.]?\d*)"}
                   ,{"Name": "ValidationScore",
                     "Regex": "###score: val_score### (\d*[.]?\d*)"}
                    ]

In [20]:
# set True if you need spot instance
use_spot = True
train_max_run_secs =   2*24 * 60 * 60
spot_wait_sec =  5 * 60
max_wait_time_secs = train_max_run_secs +  spot_wait_sec

if not use_spot:
    max_wait_time_secs = None
    s3_checkpoint=None
    sm_localcheckpoint_dir=None
    hp.pop("checkpointdir")
    
# During local mode, no spot.., use smaller dataset
if instance_type == 'local':
    use_spot = False
    max_wait_time_secs = 0
    wait = True
    # Use smaller dataset to run locally
    inputs = inputs_sample
    
    


In [21]:
job_type = "bert-sst2-classification"
base_name = "{}".format(job_type)

In [22]:
from sagemaker.pytorch import PyTorch

estimator = PyTorch(
                    entry_point='main.py',
                    source_dir = 'src',
                    role=role,
                    framework_version ="1.4.0",
                    py_version='py3',
                    instance_count=1,
                    instance_type=instance_type,
                    hyperparameters = hp,
                    output_path=s3_output_path,
                    metric_definitions=metric_definitions,
                    volume_size=30,
                    code_location=s3_code_path,
                    debugger_hook_config=False,
                    base_job_name =base_name,  
                    use_spot_instances = use_spot,
                    max_run =  train_max_run_secs,
                    max_wait = max_wait_time_secs,   
                    checkpoint_s3_uri=s3_checkpoint,
                    checkpoint_local_path=sm_localcheckpoint_dir)

estimator.fit(inputs, wait=True)

2021-01-21 19:07:26,486 - sagemaker - INFO - Creating training-job with name: bert-sst2-classification-2021-01-21-08-07-24-370
2021-01-21 08:07:27 Starting - Starting the training job...
2021-01-21 08:07:29 Starting - Launching requested ML instances...
2021-01-21 08:08:32 Starting - Preparing the instances for training......
2021-01-21 08:09:33 Downloading - Downloading input data...
2021-01-21 08:09:54 Training - Downloading the training image......
2021-01-21 08:11:23 Training - Training image download completed. Training in progress.bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2021-01-21 08:11:22,938 sagemaker-containers INFO     Imported framework sagemaker_pytorch_container.training
2021-01-21 08:11:22,964 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2021-01-21 08:11:22,969 sagemaker_pytorch_container.training INFO     Invoking user training script.
2021-01-21 08:11:23,27

2021-01-21 08:11:43,849 - transformers.file_utils - INFO - copying /tmp/tmpzqrck28z to cache at /root/.cache/torch/transformers/35d8b9d36faaf46728a0192d82bf7d00137490cd6074e8500778afed552a67e5.3fadbea36527ae472139fe84cddaa65454d7429f12d543d80bfc3ad70de55ac2
2021-01-21 08:11:44,401 - transformers.file_utils - INFO - creating metadata file for /root/.cache/torch/transformers/35d8b9d36faaf46728a0192d82bf7d00137490cd6074e8500778afed552a67e5.3fadbea36527ae472139fe84cddaa65454d7429f12d543d80bfc3ad70de55ac2
2021-01-21 08:11:44,402 - transformers.file_utils - INFO - removing temp file /tmp/tmpzqrck28z
2021-01-21 08:11:44,475 - transformers.modeling_utils - INFO - loading weights file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-cased-pytorch_model.bin from cache at /root/.cache/torch/transformers/35d8b9d36faaf46728a0192d82bf7d00137490cd6074e8500778afed552a67e5.3fadbea36527ae472139fe84cddaa65454d7429f12d543d80bfc3ad70de55ac2
2021-01-21 08:11:47,881 - transformers.modeling_utils

2021-01-21 13:20:08,860 - bert_train - INFO - Train set result details:
2021-01-21 13:20:08,902 - bert_train - INFO - Train set result details: 0.987928551277673
2021-01-21 13:20:08,902 - bert_train - INFO - Validation set result details:
2021-01-21 13:20:18,808 - bert_train - INFO - Validation set result details: 0.9220183486238532 
2021-01-21 13:20:18,809 - bert_train - INFO - Checkpoint model to /opt/ml/checkpoints/checkpoint.pt
2021-01-21 13:20:19,444 - bert_train - INFO - Run  18511     7     67352     6/8419        0% 0.004116 0.038283       0.9879       0.9220
###score: train_loss### 0.004115670372123552
###score: val_loss### 0.038282905756538615
###score: train_score### 0.987928551277673
###score: val_score### 0.9220183486238532
2021-01-21 13:58:45,962 - bert_train - INFO - Train set result details:
2021-01-21 13:58:46,003 - bert_train - INFO - Train set result details: 0.9895618346226374
2021-01-21 13:58:46,003 - bert_train - INFO - Validation set result details:
2021-01-21 13

## Deploy BERT model

#### Inference container
Ideally the server containing should already have all the required dependencies installed to reduce start up time and ensure that the runtime enviornment is consistent. This can be implemented using a custom docker image.

But for this demo, to simplify, we will let the Pytorch container script model install the dependencies during start up. As a result, you will see some of the initial ping requests fail, until all dependencies are installed.


In [23]:
from sagemaker.pytorch import PyTorchModel
from sagemaker import get_execution_role

model_uri = estimator.model_data

model = PyTorchModel(model_data=model_uri,
                     role=role,
                     py_version = "py3",
                     framework_version='1.4.0',
                     entry_point='serve.py',
                     source_dir='src')

predictor = model.deploy(initial_instance_count=1, instance_type='ml.p3.2xlarge')

2021-01-22 01:03:54,598 - sagemaker - INFO - Creating model with name: pytorch-inference-2021-01-21-14-03-54-597
2021-01-22 01:03:58,130 - sagemaker - INFO - Creating endpoint with name pytorch-inference-2021-01-21-14-03-56-185
----------------!

### Invoke API

In [24]:
import json


class TextSerDes:
    
     def serialize(self, x):
        data_bytes="\n".join(x).encode("utf-8")
        return data_bytes
    
     def deserialize(self, x, content_type):
        payload =   x.read().decode("utf-8")
        return json.loads(payload) 

In [25]:
def chunk_predict(predictor, data, chunk_size=50):
    predictor.serializer = TextSerDes()
    predictor.deserializer = TextSerDes()
    
    result = []
    for i in range(0, len(data), chunk_size):
        
        re = predictor.predict(data[i:i+chunk_size],  initial_args={ "Accept":"text/json", "ContentType" : "text/csv" })
        result.extend(re)
    return result

In [40]:
from s3_util import S3Util
import csv, io

def load_test_csv(s3_uri):
    os.path.join(processed_out_dir, "test.csv")
    data = S3Util().download_object(s3_uri).decode("utf-8")
    
    csv_reader = csv.reader(io.StringIO(data), delimiter='\t',
                            quotechar='"', quoting=csv.QUOTE_MINIMAL)
    
    actuals =[]
    inputs = []
    ids = []
    
    next(csv_reader)
    for r in  csv_reader:
        id = r[0]
        text = r[1]
        
        inputs.append(text)
        ids.append(id)
    return inputs,  ids
        

def write_predictions_csv( predictions, output_file, additional):
    
    with open(output_file, "w") as f:
        csv_writer = csv.writer(f, delimiter='\t',
                            quotechar='"', quoting=csv.QUOTE_MINIMAL)
        
        csv_writer.writerow(["prediction", "data"])
        for p,a in zip(predictions, additional):
            csv_writer.writerow([p, a])


In [41]:
%%time

test_data, ids = load_test_csv(s3_uri_test)
response  = chunk_predict(predictor, test_data )
predictions_label = [ list(l.keys())[0] for l in response ]

write_predictions_csv( predictions_label, "sst2-output.csv",test_data )

CPU times: user 198 ms, sys: 24.7 ms, total: 223 ms
Wall time: 39.8 s


In [43]:
assert len(test_data) == len(predictions_label)

## Delete endpoint

In [ ]:
predictor.delete_endpoint()

2021-01-22 09:18:58,456 - sagemaker - INFO - Deleting endpoint configuration with name: pytorch-inference-2021-01-21-14-03-56-185
2021-01-22 09:18:59,952 - sagemaker - INFO - Deleting endpoint with name: pytorch-inference-2021-01-21-14-03-56-185
